In [1]:
from app.dataprocessing.data_handler import DataHandler

data_handler = DataHandler()
data_handler.set_max_chunk_size(50)

file_path = "app/externalresources/datasets/SST_GLO_SST_L4_REP_OBSERVATIONS_010_011_slice.nc"
file_path = "app/externalresources/datasets/GEBCO_2021.nc"

data_handler.set_local_netcdf_reader(file_path)


print(f"Data structure in use: {data_handler.data_structure}")
data_handler.ds

Finished 'Loading dataset' in 0.0515 seconds
Finished 'Creating data structure' in 5.0081 seconds
Data structure in use: QuadTree with 256 chunks at lowest level of max chunk size 50


<xarray.Dataset>
Dimensions:    (lon: 86400, lat: 43200)
Coordinates:
  * lon        (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * lat        (lat) float64 -90.0 -89.99 -89.99 -89.99 ... 89.99 89.99 90.0
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 ...
Attributes: (12/36)
    title:                           The GEBCO_2021 Grid - a continuous terra...
    summary:                         The GEBCO_2021 Grid is a continuous, glo...
    keywords:                        BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ...
    Conventions:                     CF-1.6, ACDD-1.3
    id:                              DOI: 10.5285/c6612cbe-50b3-0cff-e053-6c8...
    naming_authority:                https://dx.doi.org
    ...                              ...
    geospatial_vertical_units:       meters
    geospatial_vertical_resolution:  1.0
    geospatial_vertical_positive:    up
    identifier_product_doi:          DOI: 10.5285/c6612cbe-50b3-0cff-e053-6c8...
    references:                      DOI: 10.5285/c6612cbe-50b3-0cff-e053-6c8...
    node_offset:                     1.0

In [2]:
from app.dataprocessing.benchmark import plt_img

initial_ds, bnds, node = data_handler.get_initial_ds()
#print(initial_ds)
data_variable = 'analysed_sst'
data_variable = 'elevation'
arr = initial_ds[data_variable].isel(
    {x: 0 for x in initial_ds.dims if x not in ["lat", "lon", "latitude", "longitude"]}
)
vmin_glob = arr.min().values
vmax_glob = arr.max().values

inital_render = plt_img(arr, vmin_glob, vmax_glob)

In [6]:
from ipyleaflet import basemaps, Map, ImageOverlay, projections
import ipywidgets
import numpy as np
from app.dataprocessing.benchmark import Stopwatch


def print_bounds(bounds):
    sw, ne = bounds
    s, w = sw
    n, e = ne
    return f"SW: ({s:.2f}, {w:.2f}) NE: ({n:.2f}, {e:.2f})"


# bm = basemaps.Esri.WorldTopoMap.from_qms
# bm.projections = ''
button = ipywidgets.Button(description="Render")
button2 = ipywidgets.Button(description="Clear Layers")
button3 = ipywidgets.Button(description="Render, new colors")
m = Map(crs=projections.EPSG4326, center=[0, 0], zoom=0)
m.clear_layers()  # remove default map

image = ImageOverlay(url=inital_render, bounds=((-90, -180), (90, 180)), opacity=1)
m.add_layer(image)


output = ipywidgets.Output()
with output:
    print(f"Initial render at {data_handler.get_node_resolution(node):.2f}% resolution")
    print(f"Dataset size: {data_handler.get_file_size_MB(file_path):.2f} MB, Max chunk size: {data_handler.max_chunk_size} MB")
    print(f"Bounds: {print_bounds(bnds)}")
    print(f"-----------------------------------------------------")
# display(button, button2, m, output)
display(ipywidgets.HBox((button, button3, button2)), m, output)


def handle_map_interaction(**kwargs):
    if kwargs.get("type") == "mouseup":
        with output:
            print(m.bounds)


def render(reset_color_scale=False):
    timer = Stopwatch()

    with output:
        print(f"Query bounds: {print_bounds(m.bounds)}")

    timer.start("Fetch netCDF chunk")
    sw_ne = m.bounds

    query_bounds = ((sw_ne[0][0], sw_ne[1][0]), (sw_ne[0][1], sw_ne[1][1]))
    # (np.datetime64("2021-09-23"), np.datetime64("2021-09-25")),

    file_name, bounds, xr_chunk = data_handler.request_data_netcdf(
        query_bounds, return_xr_chunk=True
    )
    fetch_time = timer.stop()

    timer.start("Rendering")
    respons_bounds_sw_ne = ((bounds[0][0], bounds[1][0]), (bounds[0][1], bounds[1][1]))
    """
    s_ = bounds[0][0].values
    w_ = bounds[1][0].values
    n_ = bounds[0][1].values
    e_ = bounds[1][1].values
    respons_bounds_sw_ne = ((float(s_), float(w_)), (float(n_), float(e_)))
    """
    data_variable = "analysed_sst"
    data_variable = "elevation"
    arr = xr_chunk.ds[data_variable].isel(
        {
            x: 0
            for x in xr_chunk.ds.dims
            if x not in ["lat", "lon", "latitude", "longitude"]
        }
    )

    if reset_color_scale:
        vmin = arr.min().values
        vmax = arr.max().values
    else:
        vmin = vmin_glob
        vmax = vmax_glob

    render = plt_img(arr, vmin, vmax)

    # change opacity on old layers

    for l in m.layers:
        if isinstance(l, ImageOverlay):
            l.opacity = 0.8

    image = ImageOverlay(url=render, bounds=respons_bounds_sw_ne, opacity=1)
    m.add_layer(image)
    render_time = timer.stop()

    with output:
        print(f"Chunk bounds: {print_bounds(respons_bounds_sw_ne)}")
        print(
            f"Chunk resolution: {data_handler.get_node_resolution(xr_chunk):.2f}%, File size: {data_handler.get_file_size_MB(file_name):.2f} MB"
        )
        print(f"{fetch_time}")
        print(f"{render_time}")
        print(f"-----------------------------------------------------")


def clear_map(event):
    m.clear_layers()


def update_map(event):
    render(reset_color_scale=False)


def reset_vmin_vmax(event):
    render(reset_color_scale=True)


button.on_click(update_map)
button2.on_click(clear_map)
button3.on_click(reset_vmin_vmax)
# m.on_interaction(handle_map_interaction)


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

Output()